#### DSPH-EXA / src/source/JSph.cpp / JSph::LoadCodeParticles

```c++
/*
        double k = 2 * M_PI / L;
        T kx = d.x[i] * k;
        T ky = d.y[i] * k;
        T kz = d.z[i] * k;       
        //Initial velocities from setting t=0 in the analytical velocity field
        d.vx[i] = A * (sin(kx - M_PI_3) * cos(ky + M_PI_3) * sin(kz + M_PI_2) -
                       cos(kz - M_PI_3) * sin(kx + M_PI_3) * sin(ky + M_PI_2));

        d.vy[i] = A * (sin(ky - M_PI_3) * cos(kz + M_PI_3) * sin(kx + M_PI_2) -
                       cos(kx - M_PI_3) * sin(ky + M_PI_3) * sin(kz + M_PI_2));

        d.vz[i] = A * (sin(kz - M_PI_3) * cos(kx + M_PI_3) * sin(ky + M_PI_2) -
                       cos(ky - M_PI_3) * sin(kz + M_PI_3) * sin(kx + M_PI_2));
*/                       

    float k_0 = 1.f;
    /* OR double k = 2 * M_PI / L; */
    velrho[p].x = A * (
      sin(k_0*pos[p].x-pi_3) * cos(k_0*pos[p].y+pi_3) * sin(k_0*pos[p].z+pi_2) -
      cos(k_0*pos[p].z-pi_3) * sin(k_0*pos[p].x+pi_3) * sin(k_0*pos[p].y+pi_2)
      );
    velrho[p].y = A * (
      sin(k_0*pos[p].y-pi_3) * cos(k_0*pos[p].z+pi_3) * sin(k_0*pos[p].x+pi_2) -
      cos(k_0*pos[p].x-pi_3) * sin(k_0*pos[p].y+pi_3) * sin(k_0*pos[p].z+pi_2)
      );
    velrho[p].z = A * (
      sin(k_0*pos[p].z-pi_3) * cos(k_0*pos[p].x+pi_3) * sin(k_0*pos[p].y+pi_2) -
      cos(k_0*pos[p].y-pi_3) * sin(k_0*pos[p].z+pi_3) * sin(k_0*pos[p].x+pi_2)
      );
    velrho[p].w = RhopZero;
```                       

In [1]:
# python3 -m ipykernel install --user --name=myvenv
import math as m
import numpy as np

In [2]:
# define CONSTANTS
p2 = m.pi / 2.  # 1.5707963267948966
p3 = m.pi / 3.  # 1.0471975511965976
k = 2 * m.pi
A = 4.*m.sqrt(2) / (3.*m.sqrt(3))

In [15]:
# infile = '/tmp/0/16/dsph/PartCsv_0000.csv.ori'
infile = '/tmp/0/16/dsph/cubeside_30/PartCsv_0000.csv'
data = np.genfromtxt(infile, delimiter=';', skip_header=1)

In [16]:
data[:, 2][0:5]

array([0.   , 0.033, 0.066, 0.099, 0.132])

In [17]:
pos_x = data[:, 0]  # numpy.ndarray
pos_y = data[:, 1]
pos_z = data[:, 2]
#
vel_x = data[:, 4]
vel_y = data[:, 5]
vel_z = data[:, 6]

np = $32^3 = 32769$

( np = $30^3 = 27000$ )

In [18]:
# len(vel_x) == 32**3
len(vel_x) == 30**3

True

In [19]:
print(f"{min(pos_x):.12f}, {min(pos_y):.12f}, {min(pos_z):.12f}" )
print(f"{max(pos_x)}, {max(pos_y)}, {max(pos_z)}" )

0.000000000000, 0.000000000000, 0.000000000000
0.95700002, 0.95700002, 0.95700002


In [20]:
# average kinetic energy per unit of mass at t=0 should be U0**2 / 2
ss_a = 0.0
for ii in range(len(vel_x)):
    ss_a += vel_x[ii]*vel_x[ii] + vel_y[ii]*vel_y[ii] + vel_z[ii]*vel_z[ii]

ss_a

26648.196738361105

In [21]:
# average kinetic energy per unit of mass at t=0 should be U0**2 / 2
ss_b = sum(vx**2 + vy**2 + vz**2 for vx, vy, vz in zip(vel_x, vel_y, vel_z))
ss_b == ss_a

True

${(v_x, v_y, v_z)_{t0}} = f(x,y,z)$

In [22]:
vel_x_from_pos = []
vel_y_from_pos = []
vel_z_from_pos = []
for ii in range(len(pos_x)):
    kx = k * pos_x[ii]
    ky = k * pos_y[ii]
    kz = k * pos_z[ii]
    #
    vel_x_from_pos.append(
        A * (
            m.sin(kx-p3) * m.cos(ky+p3) * m.sin(kz+p2) -
            m.cos(kz-p3) * m.sin(kx+p3) * m.sin(ky+p2)
        )
    )
    #
    vel_y_from_pos.append(
        A * (
            m.sin(ky-p3) * m.cos(kz+p3) * m.sin(kx+p2) -
            m.cos(kx-p3) * m.sin(ky+p3) * m.sin(kz+p2)
        )
    )
    #
    vel_z_from_pos.append(
        A * (
            m.sin(kz-p3) * m.cos(kx+p3) * m.sin(ky+p2) -
            m.cos(ky-p3) * m.sin(kz+p3) * m.sin(kx+p2)
        )
    )    

In [23]:
vel_x_from_pos == vel_x

array([False, False, False, ..., False, False, False])

In [24]:
print(vel_x_from_pos[0:3])
print(vel_x[0:3])
#
[ round(xx, 6) for xx in vel_x_from_pos[0:3] ]

[-0.9428090415820636, -1.0907010407190776, -1.1918693799691533]
[-0.94280905 -1.0907011  -1.1918694 ]


[-0.942809, -1.090701, -1.191869]

# Average kinetic energy per unit of mass
$\epsilon$ = U0**2 / 2 at t=0

In [25]:
ss_c = sum(vx**2 + vy**2 + vz**2 for vx, vy, vz in zip(vel_x_from_pos, vel_y_from_pos, vel_z_from_pos))
ss_c

26648.193555801143